In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [ ]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id='RPE_PR'
# mod_RPE_PR_old = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR_old.xml")
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Recon3D_Recon3D = read_sbml_model(Path().cwd() / "models" / "mod_Recon3D_Recon3D.xml")
# mito = read_sbml_model(Path().cwd() /"models"/  "mitocore_12918_2017_500_MOESM4_ESM.xml")
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [ ]:
# modify models

# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod.objective = 'MAR03964'
mod_Recon3D_Recon3D.objective = 'DM_atp_c__PR'

# close PR exchange reactions, open RPE exchange upper boundaries
from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
models = [mod_RPE_PR, mod_Human1_Human1, mod_Recon3D_Recon3D]
for m in models:
    # m = close_PR_EX(m)
    m = close_EX(m)
    m = open_RPE_EX_ub(m)
    for index, row in df.iterrows():  
        if row.id in [r.id for r in m.reactions]:
            m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub'])
    # m = open_RPE_EX_ub(m)


In [ ]:
# Python program to show time by process_time() 
from time import process_time
  
t1_start = process_time() 
mod_RPE_PR.optimize()
t1_stop = process_time()
   
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", t1_stop-t1_start) 


In [53]:
from src.analysis import fba_analysis, create_permutation_dicts

# set simulation parameters
# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
# 'MAR09048_RPE' O2[e_RPE] <=> 
# 'MAR04896_PR_RPE' O2[c_PR] <=> O2[e_RPE_PR]
#  MAR03964_RPE RPE ATP hydrolysis
# MAR09034_RPE(glucose exchange)

Human1_dict = create_permutation_dicts\
({'MAR09048_RPE':[(0,1000),(-2.2,1000),(-1000,1000)],\
  'MAR04896_PR_RPE':[(-1000,1000)],\
 'MAR09089_RPE':[(0,1000),(-1,1000)],\
  'MAR13037_RPE':[(0,1000),(-1,1000)],\
  'MAR13039_RPE':[(0,1000),(-1,1000)],\
  'MAR09209_RPE':[(0,1000),(-1,1000)]})

# Human1_dict2 = create_permutation_dicts({'MAR09048_RPE':[(-2.2,1000), (-1000,1000)]})                         
                                    
fba1 = fba_analysis(mod_RPE_PR,lipids_exchange_dict,'MAR03964_PR',)
fba2 = fba_analysis(mod_Human1_Human1,lipids_exchange_dict,'MAR03964_PR')

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmptyjz122y.lp
Reading time = 0.06 seconds
: 10436 rows, 28564 columns, 107712 nonzeros


C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:69: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:70: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)
C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:78: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpefkuleln.lp
Reading time = 0.13 seconds
: 16554 rows, 58114 columns, 248012 nonzeros


C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:69: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:70: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)
C:\Users\prins\GitHub\Human1_RPE-PR\src\analysis.py:78: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [50]:
    boundary_dicts = Human1_dict_p
    objective = 'MAR03964_PR'
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with mod_RPE_PR as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]

C:\Users\prins\AppData\Local\Temp\ipykernel_36080\3486918731.py:68: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_36080\3486918731.py:69: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_36080\3486918731.py:77: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


,rxn_ID,cell,lb,ub,name,subsystem,reaction,"(parsimonious flux, 1)","(parsimonious flux, 2)","(parsimonious flux, 3)","(parsimonious flux, 4)","(flux, 1)","(flux, 2)","(flux, 3)","(flux, 4)"
MAR03964_PR,MAR03964_PR,PR,0.0,1000.0,ATP phosphohydrolase,Transport reactions,ATP[c_PR] + H2O[c_PR] --> ADP[c_PR] + H+[c_PR]...,17.500139,12.500139,7.500139,2.500139,17.500139,12.500139,7.500139,2.500139
MAR12825_RPE,MAR12825_RPE,RPE,-1000.0,1000.0,uptake of pravastatin by enterocytes via proto...,Drug metabolism,H+[e_RPE] + pravastatin[e_RPE] <=> H+[c_RPE] +...,-0.290227,-0.321176,-1.684419,-1.429516,-8.371724,-1.401683,-5.642681,-3.782759
MAR12827_RPE,MAR12827_RPE,RPE,-1000.0,1000.0,pravastatin exit into portal blood,Drug metabolism,pravastatin[c_RPE] <=> pravastatin[e_RPE],-0.290227,-0.321176,-1.684419,-1.429516,-8.371724,-1.401683,-5.642681,-3.782759
MAR06916_PR,MAR06916_PR,PR,0.0,1000.0,ATP phosphohydrolase,Oxidative phosphorylation,ADP[m_PR] + 3.0 H+[i_PR] + Pi[m_PR] --> ATP[m_...,11.160012,11.160012,10.149508,8.510407,8.066268,11.012482,11.160012,10.930531
MAR05043_PR,MAR05043_PR,PR,0.0,1000.0,,Transport reactions,H+[i_PR] + Pi[c_PR] --> H+[m_PR] + Pi[m_PR],11.160012,11.160012,10.149508,8.510407,8.066268,11.012482,11.160012,10.930531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR10064_PR,MAR10064_PR,PR,0.0,1000.0,Small metabolite pool for biomass reaction,Artificial reactions,"0.0043 (S)-dihydroorotate[c_PR] + 0.0014 2,3-b...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAR10065_PR,MAR10065_PR,PR,0.0,1000.0,Cofactors and vitamins pool for biomass reaction,Artificial reactions,0.0345 11-cis-retinol[c_PR] + 0.0172 18-hydrox...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAR10067_PR,MAR10067_PR,PR,0.0,1000.0,,Inositol phosphate metabolism,"1D-myo-inositol-1,3,4,5,6-pentakisphosphate[c_...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAR10068_PR,MAR10068_PR,PR,0.0,1000.0,,Inositol phosphate metabolism,"1D-myo-inositol-1,4,5,6-tetrakisphosphate[c_PR...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
from src.get_info import make_rxn_df
df = make_rxn_df(mod_RPE_PR)
DF2 = df[df.index.isin([r.id for r in mod_RPE_PR.reactions if len(r.products)==0])]
DF2.to_clipboard()

,cell,lb,ub,ec-code,reactome,kegg.reaction,tcdb,bigg.reaction,metanetx.reaction,vmhreaction,rhea,sbo,name,subsystem,reaction,met_IDs,GPR
MAR03905_RPE,RPE,0.0,1000.0,"[1.1.1.1, 1.1.1.71]",,R00754,,ALCD2x,MNXR95725,ALCD2if,"[25291, 25290]",SBO:0000176,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NAD+[c_RPE] --> acetaldehyde[...,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ENSG00000147576 or ENSG00000172955 or ENSG0000...
MAR03907_RPE,RPE,0.0,1000.0,1.1.1.2,,R00746,,ALCD2y,MNXR95726,ALCD2yf,,SBO:0000176,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NADP+[c_RPE] --> acetaldehyde...,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ENSG00000117448
MAR04281_RPE,RPE,-1000.0,1000.0,1.1.1.27,,R00703,,,MNXR101040,r0173,,SBO:0000176,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[x_RPE] + NADH[x_RPE] + pyruvate[x_RPE] <=> ...,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04388_RPE,RPE,-1000.0,1000.0,1.1.1.27,,R00703,,LDH_L,MNXR101040,LDH_L,,SBO:0000176,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[c_RPE] + NADH[c_RPE] + pyruvate[c_RPE] <=> ...,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04283_RPE,RPE,0.0,1000.0,1.2.1.5,,R00711,,,MNXR95750,ALDD2y,"[25299, 25298]",SBO:0000176,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,acetaldehyde[c_RPE] + H2O[c_RPE] + NADP+[c_RPE...,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,ENSG00000006534 or ENSG00000108602 or ENSG0000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR10065_PR,PR,0.0,1000.0,,,,,,,,,SBO:0000185,Cofactors and vitamins pool for biomass reaction,Artificial reactions,0.0345 11-cis-retinol[c_PR] + 0.0172 18-hydrox...,0.0345 MAM00291c_PR + 0.0172 MAM00427c_PR + 0....,
MAR10067_PR,PR,0.0,1000.0,,,,,,,,,SBO:0000176,,Inositol phosphate metabolism,"1D-myo-inositol-1,3,4,5,6-pentakisphosphate[c_...",MAM00521c_PR + MAM02040c_PR --> MAM00525c_PR +...,ENSG00000151151
MAR10068_PR,PR,0.0,1000.0,,,,,,,,,SBO:0000176,,Inositol phosphate metabolism,"1D-myo-inositol-1,4,5,6-tetrakisphosphate[c_PR...",MAM00525c_PR + MAM02040c_PR --> MAM01718c_PR +...,
MAR13083_PR,PR,0.0,1000.0,,,,,,,,,SBO:0000176,,Metabolism of other amino acids,H+[c_PR] + hypotaurine[c_PR] + NADH[c_PR] + O2...,MAM02039c_PR + MAM02157c_PR + MAM02553c_PR + M...,ENSG00000010932


In [39]:
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)
    m.reactions.get_by_id('MAR09048_RPE').bounds=(-1000,1000)
    m.reactions.get_by_id('MAR09034_RPE').bounds=(-10,1000)
    m.reactions.get_by_id('MAR03964_RPE').bounds = (40,40)
    m.objective = 'MAR03964_PR'

In [41]:
mod_Human1_Human1.optimize()

,fluxes,reduced_costs
MAR03905_RPE,0.0,0.000000e+00
MAR03907_RPE,0.0,0.000000e+00
MAR04097_RPE,0.0,0.000000e+00
MAR04099_RPE,0.0,0.000000e+00
MAR04108_RPE,0.0,0.000000e+00
...,...,...
MAR10128_PR,0.0,-7.105427e-15
MAR10129_PR,0.0,-2.000800e+00
MAR10130_PR,0.0,-2.000800e+00
MAR10131_PR,0.0,-1.800080e+01


In [40]:
mod_RPE_PR.optimize()

,fluxes,reduced_costs
MAR03905_RPE,0.00,0.000000
MAR03907_RPE,0.00,0.000000
MAR04281_RPE,0.00,0.000000
MAR04388_RPE,-11.68,-0.000000
MAR04283_RPE,0.00,0.000000
...,...,...
MAR10065_PR,0.00,-24.769639
MAR10067_PR,0.00,0.000000
MAR10068_PR,0.00,0.000000
MAR13083_PR,0.00,0.000000


In [6]:
mod_RPE_PR_old.optimize()

,fluxes,reduced_costs
MAR03905_RPE,0.0,0.000000
MAR03907_RPE,0.0,0.000000
MAR04281_RPE,0.0,0.000000
MAR04388_RPE,0.0,-0.000000
MAR04283_RPE,0.0,0.000000
...,...,...
MAR10065_PR,0.0,-21.184008
MAR10067_PR,0.0,0.000000
MAR10068_PR,0.0,0.000000
MAR13083_PR,0.0,0.000000


In [11]:
mod_RPE_PR.reactions.get_by_id('MAR10023_RPE_PR')

Reaction identifier,MAR10023_RPE_PR
Name,
Memory address,0x26d34a2d9c0
Stoichiometry,MAM03970c_RPE --> MAM03971e_RPE_PR biomass --> biomass
GPR,
Lower bound,0.0
Upper bound,1000.0


In [232]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")

In [200]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Recon3D_dict = {'EX_o2s[e]_RPE':[(-10,1000)],'EX_glc_D[e]_RPE':[(-30,1000)],'DM_atp_c__RPE':[(0,0),(40,40),(80,80)]}
mitocore_dict = {'EX_o2_e':[(-1000,1000)],'EX_glc_D_e':[(-300,1000)]}

Human1_dict_p = create_permutation_dicts(Human1_dict)
Recon3D_dict_p = create_permutation_dicts(Recon3D_dict)
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

In [214]:
from src.modify_model import change_bounds
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE').bounds = (-10,1000)
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (-100,1000)

In [234]:
df = make_rxn_df(mod_RPE_PR)

In [201]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [169]:
def fba_fva_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            fva = flux_variability_analysis(m, loopless=False) #fva

            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            f_min[i] = fva['minimum']
            f_max[i] = fva['minimum']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [170]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000),(-40,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max, uptake, secretion = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:71: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:72: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:80: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
Human1_dict = create_permutation_dicts({'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0)]})
fba_fva_analysis(mod_RPE_PR, Human1_dict, 'MAR03964_PR')

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Human1_dict_p = create_permutation_dicts(Human1_dict)

fba_fva_analysis(mod_RPE_PR,Human1_dict_p, 'MAR03964_PR')